# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [2]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [3]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [4]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [5]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [7]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55563029, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [8]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [9]:
wv.most_similar('Texas')

[('Louisiana', 0.8304740190505981),
 ('Maryland', 0.8239999413490295),
 ('Pennsylvania', 0.8144728541374207),
 ('Illinois', 0.8131659626960754),
 ('Georgia', 0.8093728423118591),
 ('Ohio', 0.8069660067558289),
 ('Oregon', 0.8066929578781128),
 ('Massachusetts', 0.8019566535949707),
 ('Connecticut', 0.7988806962966919),
 ('Oklahoma', 0.782557487487793)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [10]:
wv.most_similar(negative='Texas')

[('Unabashedly', 0.41050592064857483),
 ('Per-Mile', 0.39122191071510315),
 ('DeGolia', 0.37264445424079895),
 ('Burned-Out', 0.36501967906951904),
 ('maitre', 0.36194777488708496),
 ('Hunger-Free', 0.35538625717163086),
 ('Headstrong', 0.3540554642677307),
 ('Un-Blind', 0.35391703248023987),
 ('Gatsby-Style', 0.34915533661842346),
 ('Parent/Grandparent', 0.3482239544391632)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [11]:
wv['Texas']

array([ 5.08304000e-01,  1.56964755e+00,  9.39106226e-01,  1.22560918e+00,
        3.66952240e-01, -2.63835907e-01, -8.24842691e-01, -2.30678320e-01,
       -1.23681736e+00, -3.60085535e+00,  1.59320462e+00,  6.74840629e-01,
        1.93239164e+00,  1.07044876e+00, -1.48238122e+00,  1.22913074e+00,
       -2.61581242e-01,  2.12612361e-01,  1.02091348e+00, -1.23416972e+00,
       -1.22824860e+00,  2.37260619e-03,  2.59624124e-01, -3.16092324e+00,
       -1.28893864e+00,  4.40771967e-01,  5.94388604e-01,  8.86489600e-02,
        2.74404317e-01,  8.54308307e-01,  1.14563257e-01, -3.24184924e-01,
       -4.87083882e-01, -1.63945043e+00,  4.98579979e-01,  2.99341798e+00,
       -3.22019100e-01, -1.68505251e-01,  1.40163279e+00,  2.18736529e-01,
       -2.04279199e-01, -2.84688306e+00,  1.14086366e+00,  1.77822208e+00,
        3.22305202e-01,  3.34515333e-01,  8.53204787e-01, -8.18846822e-01,
        7.10798025e-01,  4.16990328e+00,  3.62161189e-01,  1.67679310e+00,
        2.63079375e-01,  

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [12]:
wv.vectors

array([[-0.43793112, -0.2928885 ,  0.26677305, ..., -0.27787533,
        -0.8001311 ,  0.46190077],
       [ 0.9714286 ,  0.79619503,  1.6324818 , ...,  1.2160739 ,
        -1.3342451 , -0.38730916],
       [ 0.5279528 ,  1.3254124 ,  0.2820432 , ...,  2.5796359 ,
        -3.0825744 ,  0.28116912],
       ...,
       [ 0.10858278, -0.12724048, -0.06841075, ..., -0.02305965,
        -0.01800572,  0.04073293],
       [-0.04067289,  0.06191499, -0.04137627, ..., -0.0705016 ,
        -0.05871864, -0.03319494],
       [ 0.10760578, -0.14470798, -0.02332676, ..., -0.00503551,
         0.0037176 ,  0.02936325]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [15]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('princess', 0.6218807697296143),
 ('brunette', 0.6139856576919556),
 ('queen', 0.6071369051933289),
 ('villain', 0.5810404419898987),
 ('lover', 0.5682072639465332),
 ('sculpture', 0.5681293606758118),
 ('title', 0.5678552389144897),
 ('crown', 0.5653043389320374),
 ('faithful', 0.5637658834457397),
 ('subversive', 0.5565999150276184)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!